In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Con Ed Projects

In [2]:
coned_df = pd.read_csv('data/csv/coned_2020-02.csv', skiprows=2)
coned_df.head(3)

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Company,Developer,Application / Job #,Division,City/Town,Zip Code,Circuit ID,Substation,Hybrid\n(Y/N),Related Application/Job #,PV\n(kWAC),ESS\n(kWAC),WIND\n(kWAC),MT\n(kWAC),SG\n(kWAC),IG\n(kWAC),FW\n(kWAC),FC\n(kWAC),CHP\n(kWAC),GT\n(kWAC),HYDRO\n(kWAC),ICE\n(kWAC),ST\n(kWAC),OTHER\n(kWAC),Metering\n(NA / NM / RNM / CDG),Value Stack\n(Y/N),Start Date,End Date,Calculated Duration,Application Approved Date \n(Utility),Start Date (Must Match Application Approved Date),End Date.1,Calculated Duration.1,Payment Received Date,Start Date.1,End Date.2,Calculated Duration.2,CESIR cost to customer,CESIR cost to utility,Estimated Costs by Utility,Actual Customer Project Costs,Actual Utility Project Costs,Down Payment Date,Full Payment Date,Construction Start Date,Construction Complete Date,Verification Testing or Final Acceptance Date,Final Letter of Acceptance Date,Project Complete (Y/N),Utility Retention of REC (Y/N)
0,CECONY,"Sunrun Installation Services, Inc.",MC-473671,CENY-Q,Queens,11436.0,9344,Jamaica,N,NaN,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,N,2/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N
1,CECONY,"Sunrun Installation Services, Inc.",MC-473672,CENY-Q,Queens,11358.0,FL NWK,Corona_1,N,NaN,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,N,2/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N
2,CECONY,"Sunrun Installation Services, Inc.",MC-473670,CENY-Q,Queens,11429.0,SD2738,Jamaica,N,NaN,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,N,2/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N


In [3]:
# Keep only ESS, excluding thermal/ice
coned_df = coned_df[pd.notnull(coned_df['ESS\n(kWAC)'])]
coned_df = pd.DataFrame(coned_df.loc[coned_df['ICE\n(kWAC)'].isnull()])

# Drop if 'Division' = 'CENY-W'
coned_df = coned_df[coned_df.Division !='CENY-W']

# Keep columns
coned_df = pd.DataFrame(coned_df[[
    'Developer', 'Application / Job #', 'Division', 'Zip Code', 'Substation', 
    'PV\n(kWAC)', 'ESS\n(kWAC)', 'Metering\n(NA / NM / RNM / CDG)', 'Value Stack\n(Y/N)',
    'Start Date', 'Verification Testing or Final Acceptance Date', 'Final Letter of Acceptance Date', 'Project Complete (Y/N)'
]])

# Rename columns
coned_df = coned_df.rename(columns = {
    'Developer': 'developer',
    'Application / Job #': 'coned_id',
    'Division': 'borough',
    'Zip Code': 'zipcode',
    'Substation': 'substation',
    'PV\n(kWAC)': 'pv_kwac',
    'ESS\n(kWAC)': 'power_kwac',
    'Metering\n(NA / NM / RNM / CDG)': 'metering',
    'Value Stack\n(Y/N)': 'value_stack',
    'Start Date': 'application_date',
    'Verification Testing or Final Acceptance Date': 'verification_or_final_acceptance_date',
    'Final Letter of Acceptance Date': 'final_letter_of_acceptance_date',
    'Project Complete (Y/N)': 'project_complete'
})

# Rename boroughs
coned_df.loc[(coned_df.borough == 'CENY-BK'), 'borough'] = 'Brooklyn'
coned_df.loc[(coned_df.borough == 'CENY-BX'), 'borough'] = 'Bronx'
coned_df.loc[(coned_df.borough == 'CENY-M'), 'borough'] = 'Manhattan'
coned_df.loc[(coned_df.borough == 'CENY-Q'), 'borough'] = 'Queens'
coned_df.loc[(coned_df.borough == 'CENY-SI'), 'borough'] = 'Staten Island'

# Convert values to int
coned_df['zipcode'] = coned_df['zipcode'].astype(int)
coned_df['power_kwac'] = pd.to_numeric(coned_df['power_kwac'].astype(int))

# Convert to datetime
coned_df['application_date'] = pd.to_datetime(coned_df['application_date'])
coned_df['verification_or_final_acceptance_date'] = pd.to_datetime(coned_df['verification_or_final_acceptance_date'])
coned_df['final_letter_of_acceptance_date'] = pd.to_datetime(coned_df['final_letter_of_acceptance_date'])

coned_df.head(3)

,developer,coned_id,borough,zipcode,substation,pv_kwac,power_kwac,metering,value_stack,application_date,verification_or_final_acceptance_date,final_letter_of_acceptance_date,project_complete
67,"Raiden Electric, LLC",MC-473336,Brooklyn,10019,WATER_ST,NaN,2500,CDG,Y,2020-02-27,NaT,NaT,No
118,"Borrego Solar Systems, Inc.",MC-472871,Staten Island,10309,Woodrow,NaN,5000,CDG,Y,2020-02-25,NaT,NaT,No
206,Sunkeeper Solar,MC-472214,Staten Island,10007,Woodrow,NaN,5000,NaN,N,2020-02-20,NaT,NaT,No


In [4]:
# Datetime columns
coned_df['application_year'] = coned_df['application_date'].dt.strftime('%Y')
coned_df['application_month'] = coned_df['application_date'].dt.strftime('%m')
coned_df['application_ym'] = coned_df['application_date'].dt.strftime('%Y-%m')

coned_df['verification_year'] = coned_df['verification_or_final_acceptance_date'].dt.strftime('%Y')
coned_df['verification_month'] = coned_df['verification_or_final_acceptance_date'].dt.strftime('%m')
coned_df['verification_ym'] = coned_df['verification_or_final_acceptance_date'].dt.strftime('%Y-%m')

coned_df['acceptance_year'] = coned_df['final_letter_of_acceptance_date'].dt.strftime('%Y')
coned_df['acceptance_month'] = coned_df['final_letter_of_acceptance_date'].dt.strftime('%m')
coned_df['acceptance_ym'] = coned_df['final_letter_of_acceptance_date'].dt.strftime('%Y-%m')

coned_df.head(3)

,developer,coned_id,borough,zipcode,substation,pv_kwac,power_kwac,metering,value_stack,application_date,verification_or_final_acceptance_date,final_letter_of_acceptance_date,project_complete,application_year,application_month,application_ym,verification_year,verification_month,verification_ym,acceptance_year,acceptance_month,acceptance_ym
67,"Raiden Electric, LLC",MC-473336,Brooklyn,10019,WATER_ST,NaN,2500,CDG,Y,2020-02-27,NaT,NaT,No,2020,02,2020-02,NaN,NaN,NaN,NaN,NaN,NaN
118,"Borrego Solar Systems, Inc.",MC-472871,Staten Island,10309,Woodrow,NaN,5000,CDG,Y,2020-02-25,NaT,NaT,No,2020,02,2020-02,NaN,NaN,NaN,NaN,NaN,NaN
206,Sunkeeper Solar,MC-472214,Staten Island,10007,Woodrow,NaN,5000,NaN,N,2020-02-20,NaT,NaT,No,2020,02,2020-02,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Completed projects dataframe
coned_completed = coned_df[coned_df['project_complete'] == 'Yes']

# Applications dataframe
coned_apps = coned_df[coned_df['project_complete'] == 'No']

In [6]:
# Completed projects by year dataframe
coned_completed_per_year = coned_df.groupby('acceptance_year').size().reset_index().rename(columns={0: 'projects_by_year'})
coned_completed_per_year = coned_completed_per_year.rename(columns={'acceptance_year': 'year'})
print('Completed Projects by Year:')
print(coned_completed_per_year)
print()

# Overall applications per year dataframe
coned_apps_total_per_year = coned_df.groupby('application_year').size().reset_index().rename(columns={0: 'apps_total'})
coned_apps_total_per_year = coned_apps_total_per_year.rename(columns={'application_year': 'year'})
print('Applications by Year (Overall History):')
print(coned_apps_total_per_year)
print()

# Outstanding applications per year
coned_apps_pending_per_year = coned_apps.groupby('application_year').size().reset_index().rename(columns={0: 'apps_pending'})
coned_apps_pending_per_year = coned_apps_pending_per_year.rename(columns={'application_year': 'year'})
print('Pending Applications by Year:')
print(coned_apps_pending_per_year)

Completed Projects by Year:
   year  projects_by_year
0  2012                 1
1  2013                 2
2  2014                 2
3  2015                 2
4  2016                 4
5  2017                 3
6  2018                 4
7  2019                 5
8  2020                 2

Applications by Year (Overall History):
   year  apps_total
0  2012           3
1  2013           2
2  2014           9
3  2016           2
4  2017           8
5  2018           7
6  2019          38
7  2020          17

Pending Applications by Year:
   year  apps_pending
0  2016             1
1  2017             2
2  2018             3
3  2019            38
4  2020            17


In [7]:
# Merge completed and all_apps dataframes
coned_years = pd.merge(coned_completed_per_year, coned_apps_total_per_year, on='year', how='outer')
coned_years

# Merge coned_years and pending_apps dataframes
coned_years = pd.merge(coned_years, coned_apps_pending_per_year, how='outer')
coned_years

# Set year as index
coned_years.set_index('year', inplace=True)

# Total projects (running total)
coned_years['total_projects'] = coned_years.projects_by_year.cumsum()

# Rearrange columns
coned_years = coned_years[['total_projects', 'projects_by_year', 'apps_total', 'apps_pending']]

# Write CSV
coned_years.to_csv('data/clean/coned_projects.csv')
coned_years

,total_projects,projects_by_year,apps_total,apps_pending
year,,,,
2012,1,1,3.0,NaN
2013,3,2,2.0,NaN
2014,5,2,9.0,NaN
2015,7,2,NaN,NaN
2016,11,4,2.0,1.0
2017,14,3,8.0,2.0
2018,18,4,7.0,3.0
2019,23,5,38.0,38.0
2020,25,2,17.0,17.0


In [8]:
# Completed projects by borough
coned_completed_by_borough = coned_completed.groupby('borough').size().reset_index().rename(columns={0: 'completed'})
#coned_completed_by_borough.set_index('borough', inplace=False)
print('Completed Projects by Borough:')
print(coned_completed_by_borough)
print()

# Overall applications by borough
coned_apps_total_by_borough = coned_df.groupby('borough').size().reset_index().rename(columns={0: 'apps_total'})
print('Applications by Borough (Overall History):')
print(coned_apps_total_by_borough)
print()

# Pending applications by borough
coned_apps_pending_by_borough = coned_apps.groupby('borough').size().reset_index().rename(columns={0: 'apps_pending'})
print('Pending Applications by Borough:')
print(coned_apps_pending_by_borough)

Completed Projects by Borough:
         borough  completed
0          Bronx          3
1       Brooklyn          4
2      Manhattan         15
3         Queens          2
4  Staten Island          1

Applications by Borough (Overall History):
         borough  apps_total
0          Bronx           8
1       Brooklyn          32
2      Manhattan          18
3         Queens          16
4  Staten Island          12

Pending Applications by Borough:
         borough  apps_pending
0          Bronx             5
1       Brooklyn            28
2      Manhattan             3
3         Queens            14
4  Staten Island            11


In [9]:
# Merge completed and apps_total_by_borough dataframes
coned_boroughs = pd.merge(coned_completed_by_borough, coned_apps_total_by_borough, on='borough', how='outer')

# Merge new dataframe and apps_pending_by_borough
coned_boroughs = pd.merge(coned_boroughs, coned_apps_pending_by_borough, on='borough', how='outer')
coned_boroughs.set_index('borough', inplace=True)

# Write CSV
coned_boroughs.to_csv('data/clean/coned_boroughs.csv')
coned_boroughs

,completed,apps_total,apps_pending
borough,,,
Bronx,3,8,5
Brooklyn,4,32,28
Manhattan,15,18,3
Queens,2,16,14
Staten Island,1,12,11


## Power 

In [10]:
# Power by year dataframe
coned_mw_installed = coned_completed.groupby('acceptance_year', as_index=False)['power_kwac'].sum()
coned_mw_installed = coned_mw_installed.rename(columns={'acceptance_year': 'year'})
coned_mw_installed['mw_installed'] = coned_mw_installed['power_kwac'] / 1000
coned_mw_installed['mw_total_installed'] = coned_mw_installed.mw_installed.cumsum()
coned_mw_installed = coned_mw_installed.drop('power_kwac', axis=1)
print('Total MW Installed by Year:')
print(coned_mw_installed)
print()

# MW pending
coned_mw_pending = coned_apps.groupby('application_year', as_index=False)['power_kwac'].sum()
coned_mw_pending = coned_mw_pending.rename(columns={'application_year': 'year'})
coned_mw_pending['mw_pending'] = coned_mw_pending['power_kwac'] / 1000
coned_mw_pending = coned_mw_pending.drop('power_kwac', axis=1)
print('MW Pending by Year:')
print(coned_mw_pending)
print()

# MW overall applications
coned_mw_overall_apps = coned_df.groupby('application_year', as_index=False)['power_kwac'].sum()
coned_mw_overall_apps = coned_mw_overall_apps.rename(columns={'application_year': 'year'})
coned_mw_overall_apps['mw_overall_apps'] = coned_mw_overall_apps['power_kwac'] / 1000
coned_mw_overall_apps = coned_mw_overall_apps.drop('power_kwac', axis=1)
print('MW Overall Applications by Year:')
print(coned_mw_overall_apps)

Total MW Installed by Year:
   year  mw_installed  mw_total_installed
0  2012         0.050               0.050
1  2013         0.300               0.350
2  2014         0.150               0.500
3  2015         0.400               0.900
4  2016         0.400               1.300
5  2017         0.550               1.850
6  2018         0.376               2.226
7  2019         5.300               7.526
8  2020         1.200               8.726

MW Pending by Year:
   year  mw_pending
0  2016       0.100
1  2017       1.100
2  2018       1.143
3  2019     109.893
4  2020      62.100

MW Overall Applications by Year:
   year  mw_overall_apps
0  2012            0.350
1  2013            0.150
2  2014            1.200
3  2016            0.400
4  2017            1.751
5  2018            7.218
6  2019          109.893
7  2020           62.100


In [11]:
# Merge completed and pending dataframes
coned_mw = pd.merge(coned_mw_installed, coned_mw_pending, on='year', how='outer')

# Merge mw_years and mw_overall_apps
coned_mw = pd.merge(coned_mw, coned_mw_overall_apps, on='year', how='outer')
coned_mw.set_index('year', inplace=True)

# Rearrange columns
coned_mw = coned_mw[['mw_total_installed', 'mw_installed', 'mw_pending', 'mw_overall_apps']]

# Write CSV
coned_mw.to_csv('data/clean/coned_mw.csv')
coned_mw

,mw_total_installed,mw_installed,mw_pending,mw_overall_apps
year,,,,
2012,0.050,0.050,NaN,0.350
2013,0.350,0.300,NaN,0.150
2014,0.500,0.150,NaN,1.200
2015,0.900,0.400,NaN,NaN
2016,1.300,0.400,0.100,0.400
2017,1.850,0.550,1.100,1.751
2018,2.226,0.376,1.143,7.218
2019,7.526,5.300,109.893,109.893
2020,8.726,1.200,62.100,62.100
